# Assignment 5
This week we're going to carry on from last week's work with the same groups and the same underlying data, Ann Arbor parking ticket data from here: https://github.com/cab938/a2tickets

## Hints
This assignment is long. You have a big team. Reduce the assignment by getting individuals to work on portions of it, they are intentionally loosley coupled and most of your work does not need to happen on the full `DataFrame` until the end.

Also, free riders who don't do their work exist. You unfortunatly have to deal what that through social protocols, this is a team effort. I recommend meeting up on Monday at the latest to determine who didn't get things done so you can recover before the deadline.

The person in your group who has the last name closest to the beginning of the alphabet should hand in the assignment. It should include a list of the uniqunames of everyone in your group.

Finally, all of these questions assume you have some reasonably cleaned `DataFrame` from Assignment 4, so feel free to reuse that cleaning code here.

## Question 1 (10%)
Write one line of python which returns a new `DataFrame` of the average and standard deviation of the fines assessed per state.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

## Question 2 (10%)
Which parking meter makes the most money (total fine) for the city each year?

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

## Question 3 (10%)
Create a `DataFrame` which compares the make of car versus the state and calculates the average and standard deviation of fines. What conclusions might you draw from this, if any?

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

## Question 4 (10%)
My buddy says that they marked the wrong vehicle make on his ticket so he's going to get out of his fine! I think he's lying to me, but I know he's gotten tickets before. Write a query which will find any individuals in the dataset who have gotten tickets with two different makes of cars. How many people are in this set?

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

## Question 5 (10%)
Someone told me that since Michigan is home to GM and Ford, domestic cars get smaller tickets written up. I wonder if this is true? Do cars which are from GM and Ford tend to get smaller fines and penalties (look at both) than those of Honda and Toyota?

Note: this requires you to come up with some descision as to what is a GM/Ford make and what is a Honda/Toyota make. You can choose this in any reasonable way you would like to, justify your choices in your answer.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

## Question 6 (5%)
I work at 205 South State Street. Provide for me an annual breakdown of the number of tickets which are given out on this block each year in one `DataFrame`.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

## Question 7 (5%)
The license plate for my car is `DLT7389`. How much has my family generously contributed to the city of Ann Arbor in the form of fines?

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

## Question 8 (5%)
In the whole dataset, who has contributed the most to the city of Ann Arbor in the form of fines?

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

## Question 9 (15%)
It would be useful to have a column in the dataset which is called `is_vanity` which would be a `1` if the plate number was from Michigan and a vanity plate, and would be a `0` if it were from Michigan but were not a vanity plate, or was `NaN` if the vehicle was not from Michigan. Create this column using your logic (or an improvement of it) from Assignment 4, and the `Series.extract()` function.

Note: we **did not** cover this function in class, that's up to your group to figure out by reading the docs and experimenting!

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

## Question 10 (15%)
Do you think that home game weekends generate more ticket revenue for the city or less? Using the Michigan football schedules from 2016 through 2019 calculate whether home game weekends have a different level of fines from away game weekends and report your results.

Note: I'm expecting a statistical analysis, e.g. a t-test, for full points.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

## Question 11 (5% - hard)
Create a "map of shame" whereby each of my (wife's!) tickets are mapped onto a map of Ann Arbor. You can use any API/method which you think is reasonable, just describe and justify it here. 

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()